# Calibration of for LDM

In [1]:
%matplotlib widget

In [2]:
from bmcs_matmod.slide.slide_explorer import SlideExplorer
import bmcs_utils.api as bu

In [28]:
def get_step_cr(**kw):
    f_t, E_w = kw['f_t'], kw['E_w']
    bartau, E_s = kw['bartau'], kw['E_s']
    w_cr = f_t / E_w
    s_cr = bartau / E_s
    return w_cr, s_cr

In [66]:
material_params = dict(
    E_s=1, gamma_s=1, K_s=1, S_s=0.5, c_s=1, bartau=1, 
    E_w=1, S_w=0.5, c_w = 1, m = 0.05, f_t=1, f_c=-20, f_c0=-10, eta=0.0
)


(1.0, 10.0, 1.0, 10.0)

In [188]:
E_w = 18000
alpha = 0.9
E_s = alpha * E_w
E_gamma = 0 * E_s
E_K = 0
material_params = dict(
    E_s=E_s, gamma_s=E_gamma, K_s=E_K, S_s=0.0001, c_s=0.1, bartau=4, 
    E_w=E_w, S_w=0.000015, c_w = 0.1, m = 0.01, f_t=4, f_c=-100, f_c0=-20, eta=1
)

In [189]:
w_cr, s_cr = get_step_cr(**material_params)
w_max = w_cr * 50
s_max = s_cr * 50
n_steps = 1000
w_cr, w_max, s_cr, s_max

(0.00022222222222222223,
 0.011111111111111112,
 0.0002469135802469136,
 0.01234567901234568)

In [190]:
se = SlideExplorer(n_steps=n_steps, k_max=10)
se.slide_model.trait_set(**material_params);

In [191]:
calib_mtx = [
    [0, 0, w_max],
    [s_max, 0, 0],
    [s_max, 0, w_max],
    [s_max, s_max, w_max]
]

In [193]:
#%%capture
for s_x_1, s_y_1, w_1 in calib_mtx:
    print('running', s_x_1, s_y_1, w_1)
    se.reset()
    se.trait_set(s_x_1=s_x_1, s_y_1=s_y_1, w_1=w_1);
    se.run()
    bu.InteractiveWindow([se, se.energy_dissipation, se.inel_state_evolution, 
                          se.slide_model], figsize=(9,4), tight_layout=False).interact()

running 0 0 0.011111111111111112


running 0.01234567901234568 0 0


running 0.01234567901234568 0 0.011111111111111112


running 0.01234567901234568 0.01234567901234568 0.011111111111111112


In [170]:
#%%capture
se.reset()
se.trait_set(s_x_1 = s_max, s_y_1 = 0, w_1 = 0);
se.run()

In [171]:
bu.InteractiveWindow([se, se.energy_dissipation, se.inel_state_evolution, 
                      se.slide_model], figsize=(9,4), tight_layout=False).interact()

In [172]:
#%%capture
se.reset()
se.trait_set(s_x_1 = s_max, s_y_1 = s_max, w_1 = w_max);
se.run()

In [173]:
bu.InteractiveWindow([se, se.energy_dissipation, se.inel_state_evolution, 
                      se.slide_model], figsize=(9,4), tight_layout=False).interact()